## CartPole Environment

In [1]:
import gym

# Instantiate the environment
env = gym.make('CartPole-v1')

In [2]:
#It's a vector with 4 components
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
#There are two discrete actions
print("Action space:", env.action_space)

Observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Shape: (4,)
Action space: Discrete(2)


In [3]:
#The reset method is called at teh beginning of an episode
obs = env.reset()

In [4]:
#Sample a random action
action = env.action_space.sample()
print(f'Sampled action: {action}')

Sampled action: 1


In [5]:
#Step in the environment
obs, reward, terminated, info = env.step(action)

In [6]:
#Note the obs is a numpy array
#info is an empty dict for now but can contain any debugging info
#reward is a scalar
print(obs.shape, reward, terminated, info)

(4,) 1.0 False {}


### Write the function to collect data

This function collects an offline dataset of transitions that will be used to train a model using the FQI algoritm

In [7]:
from dataclasses import dataclass

import numpy as np

@dataclass
class OfflineData:
  """
  A class to store transitions
  """
  observations: np.ndarray #same as "state" in the theory
  next_observations: np.ndarray
  actions: np.ndarray
  rewards: np.ndarray
  terminateds: np.ndarray

In [8]:
def collect_data(env_id: str, n_steps: int = 50_000) -> OfflineData:
  """
  Collect transistions using a random agent (sample action randomly).

  Args:
      env_id (str): The name of the environment
      n_steps (int, optional): Number of steps to perform in the env. Defaults to 50_000.

  Returns:
      OfflineData: The collected transitions
  """

  # Create the Gym env
  env = gym.make(env_id)

  assert isinstance(env.observation_space, gym.spaces.Box)
  # Numpy arrays (buffers) to collect the data
  observations = np.zeros((n_steps, *env.observation_space.shape))
  next_observations = np.zeros((n_steps, *env.observation_space.shape))
  # Discrete actions
  actions = np.zeros((n_steps, 1))
  rewards = np.zeros((n_steps,))
  terminateds = np.zeros((n_steps,))

  obs = env.reset()

  for idx in range(n_steps):
    # Sample a random action
    action = env.action_space.sample()
    # Step in the environment
    next_obs, reward, terminated, info = env.step(action)

    # Store the transition
    observations[idx, :] = obs
    next_observations[idx, :] = next_obs
    actions[idx, :] = action
    rewards[idx] = reward
    terminateds[idx] = terminated
    # Update current observation
    obs = next_obs

    # Check if the episode is over
    # Don't forget to reset the env at the end of an episode
    if terminated:
      obs = env.reset()

  return OfflineData(
    observations,
    next_observations,
    actions,
    rewards,
    terminateds
  )


In [9]:
env_id = "CartPole-v1"
n_steps = 50_000
data = collect_data(env_id = env_id, n_steps = n_steps)

In [10]:
#Check the length of the collected data
assert len(data.observations) == n_steps
assert len(data.next_observations) == n_steps
#Check that there are multiple episodes in the data
assert not np.all(data.terminateds)
assert np.any(data.terminateds)
# Check the shape of the collected data
if env_id == "CartPole-v1":
    assert data.observations.shape == (n_steps, 4)
    assert data.next_observations.shape == (n_steps, 4)
assert data.actions.shape == (n_steps, 1)
assert data.rewards.shape == (n_steps,)

## Fitted Q Iteration (FQI) Algorithm

Fitted Q Iteration (FQI) is an algorithm that extends q-learning to continuous state space using function estimators.

It iteratively approximates the q-values for a given dataset of transitions (state, action, reward, next_state) by iteratively solving a regression problem.

Compared to later algorithms like DQN, FQI uses the whole dataset at every iteration (working on the whole batch instead of using minibatches).

In [11]:
from functools import partial
from pathlib import Path
from typing import Optional

import gym
import numpy as np
from gym import spaces
from sklearn import tree
from sklearn.base import RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

### Choosing a Model

With FQI, you can use any regression model.

Here we are choosing a k-nearest neighbors regressor, but one could choose a linear model, decision tree, a neural network, ...

In [12]:
#First choose the regressor
model_class = partial(KNeighborsRegressor, n_neighbors = 30)

### First Iteration of FQI

For $n = 0$, the initial training set is defined as:

- $x = (s_t, a_t)$
- $y = r_t$

We fit a regression model $f_\theta(x) = y$ to obtain $ Q^{n=0}_\theta(s, a) $

In [13]:
# First iteration:
# The target q-value is the reward obtained
targets = data.rewards.copy()
# Create input for current observations and actions
# Concatenate the observations and actions
# so we can predict qf(s_t, a_t)
current_obs_input = np.concatenate((data.observations, data.actions), axis = 1)
model = model_class().fit(current_obs_input, targets)

### Write the function to predict Q-values

In [14]:
def get_q_values(model: RegressorMixin, obs: np.ndarray, n_actions: int) -> np.ndarray:
  """Retrieve the q-values for a set of observations(= states in the theory).
  qf(s_t, action) for all possible actions

  Args:
      model (RegressorMixin): Q_value estimator
      obs (np.ndarray): A batch of observations
      n_actions (int): Number of discrete actions

  Returns:
      np.ndarray: The predicted q-values for the given observations (batch_size, n_actions)
  """
  
  batch_size = len(obs)
  q_values = np.zeros((batch_size, n_actions))

  for action_idx in range(n_actions):
    # Create a vector of size (batch_size, 1) for current action
    actions = action_idx * np.ones((batch_size, 1))
    # Concatentate the observations and the actions to obtain the input ot the q-value estimator
    model_input = np.concatenate((obs, actions), axis = 1)
    # Predict q-values for the given observation/action combination
    predicted_q_values = model.predict(model_input)
    # Update the q-values array for the current action
    q_values[:, action_idx] = predicted_q_values
  
  return q_values

Let's test it with a subset of the collected data

In [15]:
n_observations = 2
n_actions = int(env.action_space.n)

q_values = get_q_values(model, data.observations[:n_observations], n_actions)

assert q_values.shape == (n_observations, n_actions)

### Write the function to evaluate a model

A greedy policy $\pi(s)$ can be defined using the q-value:

$\pi(s) = argmax_{a \in A} Q(s, a)$.

It is the policy that takes the action with the highest q-value for a given state.

In [16]:
import os
from gym.wrappers import RecordVideo

def evaluate(model: RegressorMixin, env: gym.Env, n_eval_episodes: int = 10, video_name: Optional[str] = None) -> None:
  episode_returns = []
  episode_reward = 0.0
  total_episodes = 0
  done = False

  # Setup video recorder
  if video_name is not None and env.render_mode == 'rgb_array':
    os.makedirs("../logs/videos/", exist_ok = True)

    # New gym recorder always wants to cut video into episodes,
    # set video length big enough but not to inf (will cut)
    env = RecordVideo(env, "../logs/videos", step_trigger = lambda _: False, video_length = 100_000)
    env.start_recording(video_name)
  
  current_obs = env.reset()
  # Number of discrete actions
  n_actions = int(env.action_space.n)
  assert isinstance(env.action_space, spaces.Discrete), "FQI only support discrete actions"

  while total_episodes < n_eval_episodes:
    # Retrieve the q-values for the current observation
    q_values = get_q_values(model = model, obs = np.expand_dims(current_obs, axis = 0), n_actions = n_actions)
    # Select the action that maxmizes the q-value for each state
    action = np.argmax(q_values)

    current_obs, reward, terminated, info = env.step(action)

    episode_reward += float(reward)

    if terminated:
      episode_returns.append(episode_reward)
      episode_reward = 0.0
      total_episodes += 1
      current_obs = env.reset()

  if isinstance(env, RecordVideo):
    print(f"Saving video to ../logs/videos/{video_name}")
    env.close()

  print(f"Total reward = {np.mean(episode_returns):.2f} +/- {np.std(episode_returns):.2f}")

In [17]:
evaluate(model, env, n_eval_episodes = 10)

Total reward = 9.40 +/- 0.80
